## Description

Resnet Deep Learning model using Keras

## Import Libraries

In [ ]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import concatenate, add
from keras.models import Model
from keras.regularizers import l2
import keras.backend as K

from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.callbacks import TensorBoard

import numpy as np 
import json
import os
import cv2
import h5py

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from helpers import TrainingMonitor
from helpers import Utils

ImportError: cannot import name 'TrainingMonitor'

In [ ]:
output_path = "../output/"

In [ ]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)